In [1]:
# Cell 1: imports & settings
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [4]:
# Cell 2: load raw data
FILE_PATH = r"C:\Users\Aditya\OneDrive\Desktop\raj\Data analysis\Projects\Test_Dataset.csv"  
df = pd.read_csv(FILE_PATH, low_memory=False)
print("Shape:", df.shape)
df.head()

Shape: (80900, 39)


,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau
0,12202227,11250,0.00,1.00,1.00,1.00,0.00,112500,4474.8,Relative,Service,NaN,M,Male,CL,Home,0.019101,20063,2523,2318,3318,NaN,1,0,0,NaN,NaN,2.00,4.00,16.00,Yes,Yes,Self-employed,NaN,0.757150809,0.629674251,0.05,716.00,2.00
1,12279381,13500,1.00,1.00,1.00,1.00,3.00,49752,3252.15,Alone,Service,Secondary,M,Female,CL,Home,0.0105,13200,551,6972,3865,6.00,1,1,0,Laborers,5.00,3.00,3.00,18.00,No,Yes,Self-employed,NaN,0.342269008,NaN,NaN,181.00,NaN
2,12222714,38250,0.00,1.00,0.00,1.00,0.00,67500,3375,Alone,Commercial,Graduation,M,Female,RL,Home,0.032561,15943,1319,981,2296,NaN,1,0,1,Managers,2.00,1.00,2.00,19.00,Yes,Yes,Business Entity Type 3,NaN,0.683664701,NaN,0.15,281.00,NaN
3,12265215,20250,0.00,0.00,1.00,0.00,0.00,50849.55,3814.65,Alone,Govt Job,Secondary,S,Male,CL,NaN,0.01452,15634,1510,4975,6121,NaN,1,1,1,NaN,1.00,2.00,6.00,20.00,No,No,Medicine,0.35,0.229950297,0.67224289,NaN,"1,442.00",4.00
4,12203970,13500,1.00,1.00,0.00,1.00,0.00,43603.2,2515.95,Alone,Service,Secondary,M,Female,CL,Home,0.008625,15811,1075,5478,3618,7.00,1,0,0,Managers,2.00,2.00,1.00,13.00,Yes,Yes,Self-employed,0.73,0.56697104,0.73098737,NaN,927.00,1.00


In [5]:
# Cell 3: quick schema
print("Dtypes before cleaning:\n")
print(df.dtypes.sort_index())
print("\nNull counts (top 20):\n")
print(df.isna().sum().sort_values(ascending=False).head(20))


Dtypes before cleaning:

Accompany_Client               object
Active_Loan                   float64
Age_Days                       object
Application_Process_Day       float64
Application_Process_Hour      float64
Bike_Owned                    float64
Car_Owned                     float64
Child_Count                   float64
Cleint_City_Rating            float64
Client_Contact_Work_Tag        object
Client_Education               object
Client_Family_Members         float64
Client_Gender                  object
Client_Housing_Type            object
Client_Income                  object
Client_Income_Type             object
Client_Marital_Status          object
Client_Occupation              object
Client_Permanent_Match_Tag     object
Credit_Amount                  object
Credit_Bureau                 float64
Employed_Days                  object
Homephone_Tag                   int64
House_Own                     float64
ID                              int64
ID_Days                  

In [6]:
# Cell 4: standardize placeholders to NaN
PLACEHOLDERS = {"", " ", "NA", "N/A", "na", "n/a", "NaN", "-", "--"}
df = df.replace(list(PLACEHOLDERS), np.nan)

# For numeric-like columns, we'll coerce strings to numeric & treat -1 as missing (bank placeholder)
numeric_like_cols = [
    "Client_Income", "Credit_Amount", "Loan_Annuity", "Child_Count", "Client_Family_Members",
    "Population_Region_Relative", "Age_Days", "Employed_Days", "Registration_Days", "ID_Days",
    "Own_House_Age", "Cleint_City_Rating", "Score_Source_1", "Score_Source_2", "Score_Source_3",
    "Social_Circle_Default", "Phone_Change", "Credit_Bureau", "Application_Process_Day", "Application_Process_Hour"
]
for c in numeric_like_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        df.loc[df[c] == -1, c] = np.nan  # re-NULL bank placeholders


In [7]:
# Cell 5: normalize binary flags to 1/0
binary_cols = [
    "Car_Owned", "Bike_Owned", "Active_Loan", "House_Own",
    "Mobile_Tag", "Homephone_Tag", "Workphone_Working",
    "Client_Permanent_Match_Tag", "Client_Contact_Work_Tag"
]
def to_binary(s):
    if s is None or (isinstance(s, float) and np.isnan(s)): 
        return np.nan
    val = str(s).strip().lower()
    if val in {"1", "y", "yes", "true", "t"}:
        return 1
    if val in {"0", "n", "no", "false", "f"}:
        return 0
    try:
        f = float(val)
        if f in (0.0, 1.0): 
            return int(f)
    except:
        pass
    return np.nan

for c in binary_cols:
    if c in df.columns:
        df[c] = df[c].apply(to_binary)


In [8]:
# Cell 6: clean categoricals
cat_cols = [
    "Accompany_Client", "Client_Income_Type", "Client_Education", "Client_Marital_Status",
    "Client_Gender", "Loan_Contract_Type", "Client_Housing_Type", "Client_Occupation", "Type_Organization"
]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("string").str.strip()
        df[c] = df[c].replace({"": pd.NA})


In [9]:
# Cell 7: derive years from *_Days
def days_to_years(col):
    return np.where(df[col].notna(), np.round(np.abs(df[col]) / 365.25, 1), np.nan)

if "Age_Days" in df.columns:
    df["Age_Years"] = days_to_years("Age_Days")
if "Employed_Days" in df.columns:
    df["Employed_Years"] = days_to_years("Employed_Days")
if "Registration_Days" in df.columns:
    df["Registration_Years"] = days_to_years("Registration_Days")
if "ID_Days" in df.columns:
    df["ID_Years"] = days_to_years("ID_Days")


In [10]:
# Cell 8: Age buckets
def age_bucket(age_years):
    if pd.isna(age_years):
        return pd.NA
    if age_years < 25: return "<25"
    if age_years < 35: return "25–35"
    if age_years < 50: return "35–50"
    return "50+"

if "Age_Years" in df.columns:
    df["Age_Bucket"] = df["Age_Years"].apply(age_bucket)


In [11]:
# Cell 9: missingness profile
cols_to_profile = [
    "Client_Income", "Credit_Amount", "Loan_Annuity", "Client_Gender", "Age_Years",
    "Car_Owned", "Bike_Owned", "Active_Loan", "House_Own", "Client_Income_Type",
    "Client_Education", "Client_Marital_Status", "Loan_Contract_Type", "Client_Housing_Type",
    "Cleint_City_Rating", "Population_Region_Relative"
]
present_cols = [c for c in cols_to_profile if c in df.columns]
missingness = df[present_cols].isna().sum().sort_values(ascending=False).rename("Nulls").to_frame()
missingness


,Nulls
Population_Region_Relative,3278
Loan_Annuity,3208
Client_Income_Type,2501
Bike_Owned,2486
Age_Years,2484
Active_Loan,2454
Client_Income,2451
Client_Marital_Status,2443
Client_Housing_Type,2408
Car_Owned,2405


In [12]:
# Cell 10: outlier flags using Tukey IQR
def iqr_flags(series):
    s = series.dropna().astype(float)
    q1 = np.percentile(s, 25)
    q3 = np.percentile(s, 75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper

if "Client_Income" in df.columns:
    lo, hi = iqr_flags(df["Client_Income"])
    df["is_income_outlier"] = np.where(df["Client_Income"].between(lo, hi, inclusive="both") | df["Client_Income"].isna(), 0, 1)

if "Credit_Amount" in df.columns:
    lo, hi = iqr_flags(df["Credit_Amount"])
    df["is_credit_outlier"] = np.where(df["Credit_Amount"].between(lo, hi, inclusive="both") | df["Credit_Amount"].isna(), 0, 1)

df[["is_income_outlier","is_credit_outlier"]].sum(min_count=1)


is_income_outlier    3588
is_credit_outlier    1642
dtype: int64

In [13]:
# Cell 11: fraud-leaning flags (no judgment, just signals)
if {"Credit_Amount", "Client_Income"}.issubset(df.columns):
    df["flag_loan_vs_income_mismatch"] = np.where(
        (df["Credit_Amount"].notna()) & (df["Client_Income"].notna()) & (df["Credit_Amount"] > 8 * df["Client_Income"]),
        1, 0
    )
if "Employed_Years" in df.columns:
    df["flag_short_employment"] = np.where((df["Employed_Years"].notna()) & (df["Employed_Years"] < 0.5), 1, 0)

fraud_summary = {
    "loan_vs_income_mismatch": int(df.get("flag_loan_vs_income_mismatch", pd.Series(dtype=int)).sum() or 0),
    "short_employment": int(df.get("flag_short_employment", pd.Series(dtype=int)).sum() or 0),
    "total_rows": int(len(df))
}
fraud_summary


{'loan_vs_income_mismatch': 5958,
 'short_employment': 2015,
 'total_rows': 80900}

In [14]:
# Cell 12: pivot-like summaries (groupbys)

# Vehicle vs Loan
vehicle_vs_loan = (df
    .groupby(["Car_Owned","Bike_Owned"], dropna=False)
    .agg(applicants=("ID","count"),
         avg_loan_amount=("Credit_Amount","mean"),
         avg_loan_annu=("Loan_Annuity","mean"))
    .reset_index()
)
display(vehicle_vs_loan.head(10))

# Employment vs Loan
if "Client_Income_Type" in df.columns:
    employment_vs_loan = (df
        .groupby(["Client_Income_Type"], dropna=False)
        .agg(applicants=("ID","count"),
             avg_loan_amount=("Credit_Amount","mean"),
             avg_loan_annu=("Loan_Annuity","mean"))
        .reset_index()
    )
    display(employment_vs_loan.head(10))

# Age bucket vs Loan
if "Age_Bucket" in df.columns:
    cat_order = pd.CategoricalDtype(categories=["<25","25–35","35–50","50+"], ordered=True)
    tmp = df.copy()
    tmp["Age_Bucket"] = tmp["Age_Bucket"].astype(cat_order)
    agebucket_vs_loan = (tmp
        .groupby(["Age_Bucket"], dropna=False)
        .agg(applicants=("ID","count"),
             avg_loan_amount=("Credit_Amount","mean"),
             avg_loan_annu=("Loan_Annuity","mean"))
        .reset_index()
        .sort_values("Age_Bucket")
    )
    display(agebucket_vs_loan)

# Gender vs Loan
if "Client_Gender" in df.columns:
    gender_vs_loan = (df
        .groupby(["Client_Gender"], dropna=False)
        .agg(applicants=("ID","count"),
             avg_loan_amount=("Credit_Amount","mean"),
             avg_loan_annu=("Loan_Annuity","mean"))
        .reset_index()
    )
    display(gender_vs_loan)

# Region / City rating vs Loan (using Cleint_City_Rating)
if "Cleint_City_Rating" in df.columns:
    def city_segment(v):
        if pd.isna(v): return "Unknown"
        if v <= 2: return "Low Rating (<=2)"
        if v <= 4: return "Mid Rating (<=4)"
        return "High Rating (>4)"
    tmp = df.copy()
    tmp["city_segment"] = tmp["Cleint_City_Rating"].apply(city_segment)
    region_vs_loan = (tmp
        .groupby(["city_segment"], dropna=False)
        .agg(applicants=("ID","count"),
             avg_loan_amount=("Credit_Amount","mean"),
             avg_loan_annu=("Loan_Annuity","mean"))
        .reset_index()
        .sort_values("applicants", ascending=False)
    )
    display(region_vs_loan.head(10))


,Car_Owned,Bike_Owned,applicants,avg_loan_amount,avg_loan_annu
0,0.00,0.00,33719,"56,277.77","2,559.08"
1,0.00,1.00,16683,"56,962.40","2,572.92"
2,0.00,NaN,1571,"56,115.42","2,521.41"
3,1.00,0.00,17055,"66,433.51","2,987.18"
4,1.00,1.00,8618,"66,909.50","2,988.57"
5,1.00,NaN,849,"68,768.94","3,075.50"
6,NaN,0.00,1586,"60,126.19","2,730.34"
7,NaN,1.00,753,"59,275.90","2,703.46"
8,NaN,NaN,66,"64,631.95","2,807.74"


,Client_Income_Type,applicants,avg_loan_amount,avg_loan_annu
0,Businessman,2,"135,000.00","6,750.00"
1,Commercial,18237,"66,840.90","3,033.11"
2,Govt Job,5481,"66,878.14","2,884.93"
3,Retired,14199,"54,313.56","2,391.59"
4,Service,40473,"57,854.92","2,647.99"
5,Student,3,"30,575.25","1,854.90"
6,Unemployed,4,"76,837.50","3,067.99"
7,<NA>,2501,"60,006.69","2,697.53"


C:\Users\Aditya\AppData\Local\Temp\ipykernel_12700\1636694408.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Age_Bucket"], dropna=False)


,Age_Bucket,applicants,avg_loan_amount,avg_loan_annu
0,<25,3034,"42,121.94","2,187.19"
1,25–35,18413,"54,237.35","2,663.10"
2,35–50,30349,"64,604.61","2,859.90"
3,50+,26620,"60,535.91","2,623.53"
4,NaN,2484,"60,285.15","2,710.36"


,Client_Gender,applicants,avg_loan_amount,avg_loan_annu
0,Female,27126,"61,041.21","2,858.42"
1,Male,52146,"59,368.26","2,629.09"
2,XNA,1,"38,250.00","1,912.50"
3,<NA>,1627,"59,621.27","2,699.80"


,city_segment,applicants,avg_loan_amount,avg_loan_annu
0,Low Rating (<=2),68074,"60,413.43","2,730.39"
1,Mid Rating (<=4),11195,"57,102.75","2,570.68"
2,Unknown,1631,"59,376.68","2,690.57"


In [15]:
# Cell 13: save cleaned data and summaries
OUTPUT_DIR = "outputs"  # change or create if needed
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_clean_path = os.path.join(OUTPUT_DIR, "loan_clean.csv")
df.to_csv(df_clean_path, index=False)

# Export summaries if they exist
for name, obj in {
    "vehicle_vs_loan.csv": locals().get("vehicle_vs_loan"),
    "employment_vs_loan.csv": locals().get("employment_vs_loan"),
    "agebucket_vs_loan.csv": locals().get("agebucket_vs_loan"),
    "gender_vs_loan.csv": locals().get("gender_vs_loan"),
    "region_vs_loan.csv": locals().get("region_vs_loan"),
}.items():
    if obj is not None:
        obj.to_csv(os.path.join(OUTPUT_DIR, name), index=False)

print("Saved to:", os.path.abspath(OUTPUT_DIR))


Saved to: C:\Users\Aditya\OneDrive\Desktop\raj\Data analysis\Projects\outputs


In [16]:
# Cell 14: sanity checks
checks = {}
if "Credit_Amount" in df.columns:
    checks["neg_credit_amounts"] = int((df["Credit_Amount"] < 0).sum())
if "Client_Income" in df.columns:
    checks["neg_income"] = int((df["Client_Income"] < 0).sum())
if {"Loan_Annuity","Credit_Amount"}.issubset(df.columns):
    checks["annuity_gt_principal"] = int((df["Loan_Annuity"] > df["Credit_Amount"]).sum())
if "Age_Years" in df.columns:
    checks["age_out_of_bounds"] = int(((df["Age_Years"] < 18) | (df["Age_Years"] > 80)).sum())
checks


{'neg_credit_amounts': 0,
 'neg_income': 0,
 'annuity_gt_principal': 0,
 'age_out_of_bounds': 0}